In [1]:
#packages used:
#CSV, Statistics. DataFrames, Traceur, TimerOutputs, Random
#need to be installed to run the program

In [2]:
using DataFrames
using CSV
using Traceur
using TimerOutputs
using Statistics
using Random

const global randomSeed = 1
const global hit_thresh = 6 #temp
const global pseudoCount = 0.375
const global shuffles = 5
const global ALPHSIZE= 4
const global PTHRESH = 0.01
const global bgSeqFileName = "hs_chr20 - Copy.mfa"
const global motifFileName = "jaspar2009core - Copy.txt"
const global sequenceFileName = "hs_dopamine_nr.fa - Copy.txt"

"hs_dopamine_nr.fa - Copy.txt"

In [3]:
#get_motif,jl
function Get_Single_Strand_Motifs(fileName, pseudoCount)
    ssPFMs = []
    currentPFM = zeros(Float32, 0, ALPHSIZE)
    motifNames = []
    open(fileName) do file
        title = ""
        while !eof(file)
            line = readline(file)
            if(line[1:1] == ">")
                if(title == "")
                    title = line[2:length(line)]
                end
                #record what has been read so far
                if(size(currentPFM, 1) > 0)
                    push!(motifNames, title)
                    #display(currentPFM)
                    currentPFM = Normalize(currentPFM, pseudoCount)
                    push!(ssPFMs, currentPFM)
                    currentPFM = zeros(Float32, 0, ALPHSIZE)
                end
                #update the current title after the old one is recorded
                title = line[2:length(line)]
            elseif(line[1:1] == "#")
                #do nothing since this line is comment
            else
                frequencies = split(line, r"\s")
                if(length(frequencies) == ALPHSIZE) #each line should only have ALPHSIZE numbers seperated by spaces
                    frequencies = map(x->parse(Float32, x), frequencies)
                    frequencies = reshape(frequencies, 1, ALPHSIZE)
                    currentPFM = vcat(currentPFM, frequencies)
                else
                    println(line)
                    println("warning: this line has $(length(counts)) elements")
                end
                if(eof(file))
                    currentPFM = Normalize(currentPFM, pseudoCount)
                    push!(ssPFMs, currentPFM)
                    push!(motifNames, title)
                end
            end
        end
    end
    return ssPFMs, motifNames
end

function Normalize(matrix, psuedoCount)
    nrow = size(matrix, 1)
    ncol = size(matrix, 2)
    psuedoMatrix = ones(Float32, nrow, ncol) * psuedoCount
    newMatrix = matrix + psuedoMatrix
    return Normalize(newMatrix)
end

function Normalize(matrix)
    for i in 1:size(matrix, 1)
        r = sum(matrix[i,:])
        matrix[i,:] /= r
    end
    return matrix
end

function Get_Double_Strand_Motifs(singleStrandMotifs, realDoubleStrand)
    doubleStrandMotifs = []
    for motif in singleStrandMotifs
        strands = []
        temp_motif = deepcopy(motif)
        # add a column of zeros
        nrow = size(motif, 1)
        #fifthColumn = zeros(Float64, nrow, 1)
        #motif = hcat(motif, fifthColumn)
        push!(strands, motif)
        if(realDoubleStrand)
            reverseComplement = rot180(temp_motif)
            #reverseComplement = hcat(reverseComplement, fifthColumn)
            push!(strands, reverseComplement)
        end
        push!(doubleStrandMotifs, strands)
    end
    return doubleStrandMotifs
end

#(singleStrandMotifs, motifNames) = @timeit to "ss_motif" Get_Single_Strand_Motifs("test.txt", 0.375)
##display(singleStrandMotifs[1])
#doubleStrandMotifs =  @timeit to "ds_motif" Get_Double_Strand_Motifs(singleStrandMotifs, true)
#display(doubleStrandMotifs[1][2])


Get_Double_Strand_Motifs (generic function with 1 method)

In [4]:
#get_seqs.jl
function DNA_to_number(c)
    if c=='a' || c=='A'
        return UInt8(0)
    elseif c=='c' || c=='C'
        return UInt8(1)
    elseif c=='g' || c=='G'
        return UInt8(2)
    elseif c=='t' || c=='T'
        return UInt8(3)
    #else
        #return #UInt8(4)
    end
end

function number_to_DNA(i)
    if i == 0
        return 'A'
    elseif i == 1
        return 'C'
    elseif i == 2
        return 'G'
    elseif i == 3
        return 'T'
    else
        println("invalid input")
        return
    end
end

function get_fasta(fil)
    open(fil) do file
    titles=[];title="";seqs=Array{UInt8}[];seq=[]
        for s in eachline(file)
            if s[1]=='>'
                if title!=""
                    titles=vcat(titles,title)
                    push!(seqs,seq);seq=[]
                end
                title=SubString(s,2)
            elseif s[1]=='#'
                print("Comment ignored")
            else
                for x in s
                    append!(seq,DNA_to_number(x))
                end
            end
        end
        push!(titles,title)
        push!(seqs,seq)
    return seqs, titles
    end
end



get_fasta (generic function with 1 method)

In [5]:
#scan_seq.jl
struct Hit
    motif::Int64 #index of motif
    strand::Int64 #0 or 1: palindromes??
    location::Int32
    score::Float32
end

function scan_seq(seq, motif, b_probs, hitsInSequences, seqnum, motnum, hit_thresh)
    tot_score::Float32=0
    m_max::Int64 = length(motif) #m_max is the num of motifs
    b_probs = reshape(b_probs, 1, 4)
    for m in 1:m_max
        pssm = deepcopy(motif[m])
        row_max = size(motif[m], 1)

        if(row_max > length(seq))
            continue
        end

        pssm = broadcast(/, pssm, b_probs)

        #@simd for r in 1:row_max
        #    @simd for c in 1:4
        #        @fastmath pssm[r, c] /= b_probs[c]
        #    end
        #end

        #finally, scan the PSSM against the sequence:
        score::Float32 = 0
        posns = length(seq) - row_max + 1 #last possible beginning point for current motif in seq
        for n in 1:posns
            s=1
            for k in 1:row_max
                @fastmath s *= pssm[k, seq[n+k-1]+1]
            end
            score += s
            if(seqnum != -1 && log(s) >= hit_thresh)
                push!(hitsInSequences[seqnum], Hit(motnum, m, n, s))
            end
        end
        @fastmath tot_score += (score/posns/m_max)
    end
   return tot_score
end


scan_seq (generic function with 1 method)

In [6]:
#main.jl
using Random
rng= MersenneTwister();
Random.seed!(rng, randomSeed)
function get_fasta(fil)
    open(fil) do file
    titles=[];title="";seqs=Array{UInt8}[];seq=[]
        for s in eachline(file)
            if s[1]=='>'
                if title!=""
                    push!(titles,title)
                    push!(seqs,seq);seq=[]
                end
                title=SubString(s,2)
            elseif s[1]=='#'
                print("Comment ignored")
            else
                for x in s
                    if(DNA_to_number(x) != nothing)
                        append!(seq,DNA_to_number(x))
                    end
                end
            end
        end
        push!(titles,title)
        push!(seqs,seq)
    return seqs, titles
    end
end

function get_seqs(seq_file)
    myseqs=Array{UInt8}[];seq_names=[];n=""
    (myseqs,n)=get_fasta(seq_file)
    push!(seq_names,n)
    return myseqs, seq_names
    discard=pop!(myseqs)
end

function count_residues(seq, counts, ALPHSIZE)#error change seq[[]] to []
    if(counts == nothing || length(counts) < ALPHSIZE )
        counts = [0,0,0,0]
    end
    for x in 1:length(seq)
        if seq[x] < UInt(ALPHSIZE)
            counts[seq[x]+1] += 1
        end
    end
    return counts
end

function copy_masks(source, dest)
    for i in 1:length(source)
        if(source[i]==UInt8(4))
            dest[i] = UInt8(4)
        end
    end
    return dest
end

function get_base_probs(seq)
    probs = []
    counts=[]
    #count_residues(seq, counts, alphsize)
    temp_counts = count_residues(seq, counts, ALPHSIZE)
    counts  = temp_counts
    tot=0
    for x in counts
        tot=tot+x
    end
    #println("counts in get_base_probs: ", counts)
    for i = 1:ALPHSIZE
        push!(probs, counts[i]/tot)
    end
    return probs
end


mutable struct result
    motifIndex::Int64
    rawScore::Float32
    pValues::Array{Float32, 1}
    seqScores::Array{Float32, 1}
end

mutable struct seq_info
    num::Int64
    len::Int64
    gc::Float32
end
function init_seq_info(seqs)#seqs:[[]]
    num = length(seqs)
    len=0
    for s = 1:num
        len=len+length(seqs[s])
    end
    counts=[]
    for s = 1:length(seqs)
        temp_counts = count_residues(seqs[s], counts, ALPHSIZE)
        counts  = temp_counts
    end
    # println("in init_seq_info, counts: $counts")
    tot=0
    for x in counts
        tot=tot+x
    end
    gc = (counts[2]+counts[3])/tot
    return seq_info(num, len, gc)
end

function bg_fragment(bg_seqs, frag, len, frag_num, frag_tot)
    if(length(bg_seqs) == 0)
        return
    end
    b = 1 #select which bg seq
    r = rand(rng,1:frag_tot)
    for b in 1:length(bg_seqs)
        if(frag_num[b]>r)
            break
        end
        r -= frag_num[b]
    end

    #if(b == length(bg_seqs))
    #    return
    #end

    # p =  Vector{UInt}()
    posns = length(bg_seqs[b]) - len + 1
    #p = bg_seqs[b][1]+rand(rng,1:posns)
    p = rand(rng,1:posns)
    println("bgfrag starting position: ", p)
    # we don't support the bg_seq having any masked character yet so I think this is not necessary
    #flag=true
    #while (flag)
    #    p = bg_seqs[b][1]+rand(rng,1:posns-1)
    #    ind = 0
    #    # println(p," ",len)
    #    for i = p:p+len
    #        if bg_seqs[b][i] == ALPHSIZE
    #            ind=i
    #            break
    #        end
    #        if i==p+len
    #            ind=i
    #        end
    #    end
    #    if ind == p+len
    #        flag= false
    #    end
    #end

    # push!(p,bg_seqs[b][1]+rand(1:posns))
    # for i in 1:length(p)+len
    #     if(bg_seqs[b][i]==UInt8(4)&& i!= length(p)+len)
    #         push!(p,bg_seqs[b][1]+rand(1:posns))
    #     end
    # end

    for i in p:p+len-1
        push!(frag,bg_seqs[b][i])
    end
    #println("frag: $frag")
    return
end

function is_significant(r)
    for p = 1:length(r.pValues)
        if (r.pValues[p] > PTHRESH && r.pValues[p] < 1-PTHRESH)
            return false
        end
    end
    return true
end

is_significant (generic function with 1 method)

In [7]:
#combine_score.jl
function combine_score(scores)
    n = length(scores)
    prevCol = zeros(Float32, 1, n)

    for j in 1:n
        prevCell = -1
        currentCell = -1
        for i in 1:j
            if(i == 1)
                currentCell = (scores[j] + (j-i)*prevCol[i])/j
            else
                prevCell = currentCell
                currentCell = (i*scores[j]*prevCol[i-1] + (j-i)*prevCol[i])/j
                prevCol[i-1] = prevCell
            end
            #println("prevCell at[$i, $j]: $prevCell")
        end
        prevCol[j] = currentCell
    end
    return mean(prevCol)
end

combine_score (generic function with 1 method)

In [8]:
#generate_seq.jl
rng= MersenneTwister();
Random.seed!(rng, randomSeed)
# t - no of dna samples, m - motif, l - length of each sample in dna
function dna_generator(t,m,l)
    out = Array{String}(undef,t)
    if length(m)>l
        print("Motif can't be longer than the length of DNA\n")
    else
        gc=0;
        if 0.41*l-floor(0.41*l)>0.5
            req=floor(0.41*l)+1
        else
            req=floor(0.41*l)
        end
        for x in m
            if x=='C' || x=='G'
                gc=gc+1
            end
        end
        if gc>=req
            for i = 1:t
                tem = "";len=l-length(m)
                j=rand(rng,big.(1:len))
                while len>=0
                    if len==j
                        tem=string(tem,m)
                    else
                        tem=string(tem,rand(rng,['A','T']))
                    end
                    len=len-1
                end
                out[i] = tem
            end
        else
           for i = 1:t
                tem = "";l1=req-gc;len=l-length(m)-l1
                while l1>0
                    tem=string(tem,rand(rng,['C','G']))
                    l1=l1-1
                end
                j=rand(rng,big.(1:len))
                while len>=0
                    if len==j
                        tem=string(tem,m)
                    else
                        tem=string(tem,rand(rng,['A','T']))
                    end
                    len=len-1
                end
                out[i] = tem
            end
        end
        return out
    end
end

function make_fasta(input,file)
    open(file, "w") do io
        for x in 1:length(input)
            write(io, "> Generated Sequence "*string(x)*"\n")
            write(io, input[x]*"\n")
        end
    end
end

#make_fasta(dna_generator(30,"ACACTGTAAACTGCTACTACTGGGCACTGTGG",400),"input_seq.txt")
#make_fasta(dna_generator(70,"ACTTTGGGAACTGCTAACACCACACCGGGTTTGGCACTGACCTACTGACTGTAGGTGG",400),"input_bg.txt")


make_fasta (generic function with 1 method)

In [9]:
#get_raw_score.jl
using TimerOutputs
function print_result(sequenceFileName, motifFileName, seq_info, motifSize, results)
    println("Sequence file: $sequenceFileName ($(seq_info.num) sequences, $(seq_info.len) bp, $(seq_info.gc) GC content)")
    println("Motif file: $motifFileName ($motifSize motifs)")

    for i in 1:length(results)
        results[i].motifIndex = i
    end
    sort!(results, by = x -> x.rawScore)

    println("\n ************* Over and under represented motifs *************")
    summary = DataFrame(Motif = String[], RawScore = Float64[], PValue = String[])
    for i in 1:length(results)
        if(is_significant(results[i]))
            motifName = motifNames[results[i].motifIndex]
            motifRawScore = results[i].rawScore
            pValueAsString = ""
            for j in results[i].pValues
                pValueAsString = string(pValueAsString, string(j))
            end
            push!(summary, [motifName, log(motifRawScore), pValueAsString])
        end

    end
    show(summary)
    CSV.write("summary.csv", summary)

    println("*** Motif Instances with Score >= 6:")
    for i in 1:length(hitsInSequences)
        hitFrame = DataFrame(Motif = String[], Location = String[], Strand = String[], Sequence = String[], Score = Float64[])
        println("hitsInSequence $i length: $(length(hitsInSequences[i]))")
        for j in 1:length(hitsInSequences[i])
            hit = hitsInSequences[i][j]
            strand = ""
            if(hit.strand == 1)
                strand = "+"
            elseif(hit.strand == 2)
                strand = "-"
            else
                print("Something wrong here! hit#: $j, sequence#: $i")
            end

            motifWidth = size(singleStrandMotifs[hit.motif],1)
            location = String("$(hit.location) - $(hit.location + motifWidth - 1)")
            motifString = ""
            for w in 0:(motifWidth-1)
                motifString = string(motifString, number_to_DNA(sequence[i][(hit.location + w)]))
            end
            push!(hitFrame, [motifNames[hitsInSequences[i][j].motif], location, strand, motifString, log(hit.score)])
        end
        show(hitFrame)
        println("")
        fileName = string("hitInSequence", i)
        fileName = string(fileName, ".csv")
        CSV.write(fileName, hitFrame)
    end

    println("Background sequence file: $bgSeqFileName")
    println("Randomizations: $shuffles")
    println("P-value threshhold: $PTHRESH")
    println("Motif score threshhold: $hit_thresh")
    println("Pseudocount: $pseudoCount")
    println("Random seed: $randomSeed")
end

function shuffle_bgseq(seqs,bg_seqs,ds_motifs,results)
    fragNums = []
    println("Analyzing background sequences...")
    for i in 1:length(seqs)
        t=[]
        for j in 1:length(bg_seqs)
            frags=0
            r=length(bg_seqs[j])
            if(r > length(seqs[i]))
                frags = r - length(seqs[i])
            end
            #@simd for x in 1:length(bg_seqs[j])
                # global fragNums
                #if bg_seqs[j][x]==ALPHSIZE
                #    r=0
                #else
                    #r=r+1
                    #if r>length(seqs[i])
                    #    frags=frags+1
                    #end
                #end
            #end
            push!(t,frags)
        end
        push!(fragNums,t)
    end
    # println("computed frag_num")
    frag_tots = []
    for x in 1:length(fragNums)
        #tot=0
        tot = sum(fragNums[x])
        #for y in 1:length(fragNums[x])
        #    tot=tot+fragNums[x][y]
        #end
        if tot==0
            print("\nDie function : Can't get fragments of control sequences to match all target sequences.")
        end
        push!(frag_tots,tot)
    end
    # println("computed frag_tots")
    losses = zeros(length(ds_motifs))
    pValues = []#Vector{Float64}(0,length(ds_motifs))
    @simd for r = 1:shuffles
        println("The $r th shuffle")
        r_seqs = []#Vector{Int64}
        b_probs = []#Vector{Float64}
        @simd for s = 1:length(seqs)
            temp_seqs = []
            @timeit to "bg_fragment" bg_fragment(bg_seqs, temp_seqs, length(seqs[s]), fragNums[s], frag_tots[s])
            push!(r_seqs,temp_seqs)
            r_seqs[s] = @timeit to "copy_masks" copy_masks(seqs[s], r_seqs[s])
            temp_probs = @timeit to "get_base_probs" get_base_probs(r_seqs[s])
            push!(b_probs, temp_probs)
        end
        @timeit to "rand_test" rand_test(r_seqs, b_probs, ds_motifs, losses)
    end

    for m = 1:length(ds_motifs)
        push!(results[m].pValues, losses[m]/shuffles)
    end
    # return pValues
end

function rand_test(myseqs, b_probs, motifs, losses)
    for m in 1:length(motifs)
        scores = Vector{Float64}()
        for s in 1:length(myseqs)
            push!(scores,@timeit to "scan_seq" scan_seq(myseqs[s], motifs[m], b_probs[s], hitsInSequences, -1, m, 6))
        end
        raw = @timeit to "combine_score" combine_score(scores)
        if (raw >= results[m].rawScore)#notes:no result[m]
          losses[m]+=1;
      end
    end
end

function get_hits(seqs,ds_motifs,base_probs,results,hits)
    resize!(hits,length(seqs))
    for m in 1:length(ds_motifs)
        if (is_significant(results[m]))
            for s in 1:length(seqs)
                scan_seq(seqs[s], ds_motifs[m], base_probs[s], hitsInSequences, s, m, hit_thresh)
            end
        end
    end
end




Random.seed!(randomSeed)
bg_info=[]
bg_seqs=[]
bg_sequenceNames=[]
const global to = TimerOutput()
(bg_seqs, junk) = @timeit to "get_seqs" get_seqs(bgSeqFileName)
(singleStrandMotifs, motifNames) = @timeit to "ss_motif" Get_Single_Strand_Motifs(motifFileName, pseudoCount)
#display(singleStrandMotifs[1])
doubleStrandMotifs =  @timeit to "ds_motif" Get_Double_Strand_Motifs(singleStrandMotifs, true)
#display(doubleStrandMotifs[1])
sequenceNames = []
sequence = []
(sequence, sequenceNames) = @timeit to "get_seqs" get_seqs(sequenceFileName)
#println(sequence[1])
seqInfo = init_seq_info(sequence)
base_probs = []
for s in 1:length(sequence)
    temp_base_probs = get_base_probs(sequence[s])
    push!(base_probs, temp_base_probs)
end

#dp=Array{Float64}(undef,length(sequence[1])+1)

results = Vector{result}(undef,length(doubleStrandMotifs))

hitsInSequences = [Vector{Hit}() for i in 1:length(sequence)]
for m in 1:length(doubleStrandMotifs)
    sequenceScores = []
    #println("scanning motif $m")
    for s in 1:length(sequence)
        a = scan_seq(sequence[s], doubleStrandMotifs[m], base_probs[s], hitsInSequences, -1, m, hit_thresh)
        push!(sequenceScores, a)
    end
    rawScore = combine_score(sequenceScores)
    #println("motif $m: ", rawScore)
    results[m] = result(0, rawScore, [], sequenceScores)
end

push!(bg_info,init_seq_info(bg_seqs))
println("starting to shuffle!")
p_vals = @timeit to "shuffle bg_seq" shuffle_bgseq(sequence, bg_seqs, doubleStrandMotifs, results)

@timeit to "get_hits" get_hits(sequence, doubleStrandMotifs, base_probs, results, hitsInSequences)
print_result(sequenceFileName, motifFileName, seqInfo, length(doubleStrandMotifs), results)
to_flat=TimerOutputs.flatten(to);
show(to_flat)






starting to shuffle!
Analyzing background sequences...
The 1 th shuffle
bgfrag starting position: 1533059
bgfrag starting position: 1569986
The 2 th shuffle
bgfrag starting position: 804079
bgfrag starting position: 1140412
The 3 th shuffle
bgfrag starting position: 1700637
bgfrag starting position: 998171
The 4 th shuffle
bgfrag starting position: 245188
bgfrag starting position: 579851
The 5 th shuffle
bgfrag starting position: 1180024
bgfrag starting position: 931407
Sequence file: hs_dopamine_nr.fa - Copy.txt (2 sequences, 3002 bp, 0.57195204 GC content)
Motif file: jaspar2009core - Copy.txt (126 motifs)

 ************* Over and under represented motifs *************
39×3 DataFrame
│ Row │ Motif                  │ RawScore  │ PValue │
│     │ String                 │ Float64   │ String │
├─────┼────────────────────────┼───────────┼────────┤
│ 1   │ MA0066.1 PPARG         │ -4.59891  │ 1.0    │
│ 2   │ MA0083.1 SRF           │ -3.75833  │ 0.0    │
│ 3   │ MA0085.1 Su(H)         │ -2